In [1]:
import random

import pandas as pd
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adam
from keras.initializers import VarianceScaling
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score


Using TensorFlow backend.


In [2]:
df = pd.read_csv("./data/lgb.csv")
df

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,...,userid_in_feedid_nunique,feedid_in_userid_nunique,userid_in_authorid_nunique,authorid_in_userid_nunique,userid_authorid_count,userid_in_authorid_count_prop,authorid_in_userid_count_prop,videoplayseconds_in_userid_mean,videoplayseconds_in_authorid_mean,feedid_in_authorid_nunique
0,8,71474,1,1,0.0,0.0,1.0,500.0,5366.0,0.0,...,394,260,510,190,2,0.003105,0.007664,12642.308,10255.0550,5
1,8,73916,1,1,0.0,0.0,0.0,250.0,1533.0,0.0,...,1429,260,1429,190,1,0.000683,0.003832,12642.308,16000.0000,1
2,8,50282,1,1,0.0,0.0,0.0,750.0,1302.0,0.0,...,2492,260,2995,190,1,0.000265,0.003832,12642.308,28736.4940,24
3,8,11391,1,1,0.0,0.0,1.0,3750.0,5191.0,0.0,...,299,260,1207,190,3,0.001360,0.011500,12642.308,6875.7370,17
4,8,27349,1,1,0.0,0.0,0.0,250.0,800.0,0.0,...,29,260,277,190,2,0.004080,0.007664,12642.308,17253.5780,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7739862,133812,56450,15,2,NaN,NaN,NaN,NaN,NaN,NaN,...,149,274,237,177,1,0.004080,0.003597,17314.080,14221.3120,3
7739863,231669,76501,15,1,NaN,NaN,NaN,NaN,NaN,NaN,...,90,223,628,161,2,0.002080,0.008930,16950.672,6608.3335,19
7739864,179168,70550,15,2,NaN,NaN,NaN,NaN,NaN,NaN,...,385,744,4083,548,2,0.000263,0.002659,26476.697,37323.2150,86
7739865,92546,49432,15,2,NaN,NaN,NaN,NaN,NaN,NaN,...,811,282,6013,203,4,0.000294,0.013750,52482.758,13586.7250,18


In [3]:
train = df[~df['read_comment'].isna()].reset_index(drop=True)
test = df[df['read_comment'].isna()].reset_index(drop=True)
trn_x = train[train['date_'] < 14].reset_index(drop=True)
val_x = train[train['date_'] == 14].reset_index(drop=True)

In [4]:
play_cols = ['is_finish', 'play_times', 'play', 'stay']
y_list = ['read_comment', 'like', 'click_avatar', 'forward', 'favorite', 'comment', 'follow']
max_day = 15
cols = [f for f in df.columns if f not in ['date_'] + play_cols + y_list]

In [5]:
# trn_x = trn_x[cols]
print(trn_x.columns)
trn_x

Index(['userid', 'feedid', 'date_', 'device', 'read_comment', 'comment',
       'like', 'play', 'stay', 'click_avatar', 'forward', 'follow', 'favorite',
       'authorid', 'videoplayseconds', 'is_finish', 'play_times',
       'userid_5day_count', 'userid_5day_finish_rate',
       'userid_5day_play_times_max', 'userid_5day_play_times_mean',
       'userid_5day_play_max', 'userid_5day_play_mean', 'userid_5day_stay_max',
       'userid_5day_stay_mean', 'userid_5day_read_comment_sum',
       'userid_5day_read_comment_mean', 'userid_5day_like_sum',
       'userid_5day_like_mean', 'userid_5day_click_avatar_sum',
       'userid_5day_click_avatar_mean', 'userid_5day_forward_sum',
       'userid_5day_forward_mean', 'feedid_5day_count',
       'feedid_5day_finish_rate', 'feedid_5day_play_times_max',
       'feedid_5day_play_times_mean', 'feedid_5day_play_max',
       'feedid_5day_play_mean', 'feedid_5day_stay_max',
       'feedid_5day_stay_mean', 'feedid_5day_read_comment_sum',
       'feedid_5d

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,...,userid_in_feedid_nunique,feedid_in_userid_nunique,userid_in_authorid_nunique,authorid_in_userid_nunique,userid_authorid_count,userid_in_authorid_count_prop,authorid_in_userid_count_prop,videoplayseconds_in_userid_mean,videoplayseconds_in_authorid_mean,feedid_in_authorid_nunique
0,8,71474,1,1,0.0,0.0,1.0,500.0,5366.0,0.0,...,394,260,510,190,2,0.003105,0.007664,12642.308,10255.055,5
1,8,73916,1,1,0.0,0.0,0.0,250.0,1533.0,0.0,...,1429,260,1429,190,1,0.000683,0.003832,12642.308,16000.000,1
2,8,50282,1,1,0.0,0.0,0.0,750.0,1302.0,0.0,...,2492,260,2995,190,1,0.000265,0.003832,12642.308,28736.494,24
3,8,11391,1,1,0.0,0.0,1.0,3750.0,5191.0,0.0,...,299,260,1207,190,3,0.001360,0.011500,12642.308,6875.737,17
4,8,27349,1,1,0.0,0.0,0.0,250.0,800.0,0.0,...,29,260,277,190,2,0.004080,0.007664,12642.308,17253.578,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708841,250236,72813,12,2,0.0,0.0,0.0,6369.0,6566.0,0.0,...,1068,186,2812,172,1,0.000169,0.005207,21958.115,20080.752,40
6708842,250236,52019,12,2,0.0,0.0,0.0,13212.0,13708.0,0.0,...,750,186,750,172,1,0.001310,0.005207,21958.115,11000.000,1
6708843,250236,51045,12,2,0.0,0.0,0.0,0.0,32215.0,0.0,...,112,186,348,172,1,0.001767,0.005207,21958.115,30656.637,16
6708844,250236,46881,12,2,0.0,0.0,0.0,0.0,5618.0,0.0,...,989,186,3216,172,1,0.000236,0.005207,21958.115,14078.376,18


In [6]:
label_columns = y_list[:4]
trn_x_raw_labels = trn_x[label_columns]
val_x_raw_labels = val_x[label_columns]

In [7]:
# transformed_train = pd.get_dummies(train_df.drop(label_columns, axis=1), columns=categorical_columns)
# transformed_other = pd.get_dummies(other_df.drop(label_columns, axis=1), columns=categorical_columns)

# Filling the missing column in the other set
# transformed_other['det_hh_fam_stat_ Grandchild <18 ever marr not in subfamily'] = 0

# One-hot encoding categorical labels
# ['read_comment', 'like', 'click_avatar', 'forward'
trn_x_read_comment = to_categorical((trn_x_raw_labels.read_comment != 0.0).astype(int), num_classes=2)
trn_x_like = to_categorical((trn_x_raw_labels.like != 0.0).astype(int), num_classes=2)
trn_x_click_avatar = to_categorical((trn_x_raw_labels.click_avatar != 0.0).astype(int), num_classes=2)
trn_x_forward = to_categorical((trn_x_raw_labels.forward != 0.0).astype(int), num_classes=2)

val_x_read_comment = to_categorical((val_x_raw_labels.read_comment != 0.0).astype(int), num_classes=2)
val_x_like = to_categorical((val_x_raw_labels.like != 0.0).astype(int), num_classes=2)
val_x_click_avatar = to_categorical((val_x_raw_labels.click_avatar != 0.0).astype(int), num_classes=2)
val_x_forward = to_categorical((val_x_raw_labels.forward != 0.0).astype(int), num_classes=2)

# other_income = to_categorical((other_raw_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
# other_marital = to_categorical((other_raw_labels.marital_stat == ' Never married').astype(int), num_classes=2)

dict_outputs = {
    'read_comment': trn_x_read_comment.shape[1],
    'like': trn_x_like.shape[1],
    # 'click_avatar': trn_x_click_avatar.shape[1],
    # 'forward': trn_x_forward.shape[1]
}
dict_train_labels = {
    'read_comment': trn_x_read_comment,
    'like': trn_x_like,
    # 'click_avatar': trn_x_click_avatar,
    # 'forward': trn_x_forward,
}
dict_other_labels = {
    'read_comment': val_x_read_comment,
    'like': val_x_like,
    # 'click_avatar': val_x_click_avatar,
    # 'forward': val_x_forward,
}
output_info = [(dict_outputs[key], key) for key in sorted(dict_outputs.keys())]

# Split the other dataset into 1:1 validation to test according to the paper
trn_x = trn_x[cols].fillna(0)
val_x = val_x[cols].fillna(0)
validation_indices = val_x.sample(frac=0.5, replace=False, random_state=1234).index
test_indices = list(set(val_x.index) - set(validation_indices))
validation_data = val_x.iloc[validation_indices]
validation_label = [dict_other_labels[key][validation_indices] for key in sorted(dict_other_labels.keys())]
test_data = val_x.iloc[test_indices]
test_label = [dict_other_labels[key][test_indices] for key in sorted(dict_other_labels.keys())]
train_data = trn_x
train_label = [dict_train_labels[key] for key in sorted(dict_train_labels.keys())]


In [9]:
from mmoe import MMoE
def main():
    # Load the data
    num_features = train_data.shape[1]

    print('Training data shape = {}'.format(train_data.shape))
    print('Validation data shape = {}'.format(validation_data.shape))
    print('Test data shape = {}'.format(test_data.shape))

    # Set up the input layer
    input_layer = Input(shape=(num_features,))

    # Set up MMoE layer
    mmoe_layers = MMoE(
        units=4,
        num_experts=8,
        num_tasks=2
    )(input_layer)

    output_layers = []

    # Build tower layer from MMoE layer
    for index, task_layer in enumerate(mmoe_layers):
        tower_layer = Dense(
            units=8,
            activation='relu',
            kernel_initializer=VarianceScaling())(task_layer)
        output_layer = Dense(
            units=output_info[index][0],
            name=output_info[index][1],
            activation='softmax',
            kernel_initializer=VarianceScaling())(tower_layer)
        output_layers.append(output_layer)

    # Compile model
    model = Model(inputs=[input_layer], outputs=output_layers)
    adam_optimizer = Adam()
    model.compile(
        loss={'like': 'binary_crossentropy', 'read_comment': 'binary_crossentropy'},
        optimizer=adam_optimizer,
        metrics=['accuracy']
    )

    # Print out model architecture summary
    model.summary()

    # Train the model
    model.fit(
        x=train_data,
        y=train_label,
        validation_data=(validation_data, validation_label),
        callbacks=[
            ROCCallback(
                training_data=(train_data, train_label),
                validation_data=(validation_data, validation_label),
                test_data=(test_data, test_label)
            )
        ],
        epochs=1
    )


if __name__ == '__main__':
    main()


Training data shape = (6708846, 82)
Validation data shape = (304518, 82)
Test data shape = (304518, 82)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 82)           0                                            
__________________________________________________________________________________________________
m_mo_e_1 (MMoE)                 [(None, 4), (None, 4 3984        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            40          m_mo_e_1[0][0]                   
__________________________________________________________________________________________________
dense

KeyboardInterrupt: 

In [8]:
class ROCCallback(Callback):
    def __init__(self, training_data, validation_data, test_data):
        self.train_X = training_data[0]
        self.train_Y = training_data[1]
        self.validation_X = validation_data[0]
        self.validation_Y = validation_data[1]
        self.test_X = test_data[0]
        self.test_Y = test_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        train_prediction = self.model.predict(self.train_X)
        validation_prediction = self.model.predict(self.validation_X)
        test_prediction = self.model.predict(self.test_X)

        # Iterate through each task and output their ROC-AUC across different datasets
        for index, output_name in enumerate(self.model.output_names):
            train_roc_auc = roc_auc_score(self.train_Y[index], train_prediction[index])
            validation_roc_auc = roc_auc_score(self.validation_Y[index], validation_prediction[index])
            test_roc_auc = roc_auc_score(self.test_Y[index], test_prediction[index])
            print(
                'ROC-AUC-{}-Train: {} ROC-AUC-{}-Validation: {} ROC-AUC-{}-Test: {}'.format(
                    output_name, round(train_roc_auc, 4),
                    output_name, round(validation_roc_auc, 4),
                    output_name, round(test_roc_auc, 4)
                )
            )

        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
